In [ ]:
## Task 1: Dataset Preparation and Preprocessing
### 1. Load the MRI Brain Scan Image

import cv2
import matplotlib.pyplot as plt

def load_mri_image_22bai1118(image_path_22bai1118):
    return cv2.imread(image_path_22bai1118, cv2.IMREAD_GRAYSCALE)

def display_image_22bai1118(image_22bai1118, title_22bai1118):
    plt.imshow(image_22bai1118, cmap='gray')
    plt.title(title_22bai1118)
    plt.axis('off')
    plt.show()

image_path_22bai1118 = '/kaggle/input/dataaa/image.jpeg'

image_22bai1118 = load_mri_image_22bai1118(image_path_22bai1118)
display_image_22bai1118(image_22bai1118, "Original MRI Image")
### 2. Apply Noise Reduction and Contrast Enhancement
def preprocess_image_22bai1118(image_22bai1118):
    smoothed_22bai1118 = cv2.GaussianBlur(image_22bai1118, (5, 5), 0)
    
    clahe_22bai1118 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced_22bai1118 = clahe_22bai1118.apply(smoothed_22bai1118)
    
    return enhanced_22bai1118

enhanced_22bai1118 = preprocess_image_22bai1118(image_22bai1118)

def display_preprocessed_22bai1118(image_22bai1118, title_22bai1118):
    plt.figure(figsize=(12, 6))
    plt.imshow(image_22bai1118, cmap='gray')
    plt.title(title_22bai1118)
    plt.axis('off')
    plt.show()

display_preprocessed_22bai1118(enhanced_22bai1118, "Preprocessed Image")

## Task 2: Watershed Algorithm Implementation
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters
from skimage.segmentation import watershed
from skimage import morphology as morph

preprocessed_22bai1118 = preprocess_image_22bai1118(image_22bai1118)

def create_tumor_markers_22bai1118(image_22bai1118):
    # Apply adaptive thresholding
    _, binary_22bai1118 = cv2.threshold(image_22bai1118, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphological operations to refine tumor region
    kernel_22bai1118 = np.ones((3,3), np.uint8)
    opening_22bai1118 = cv2.morphologyEx(binary_22bai1118, cv2.MORPH_OPEN, kernel_22bai1118, iterations=2)
    
    # Distance transform
    dist_transform_22bai1118 = cv2.distanceTransform(opening_22bai1118, cv2.DIST_L2, 5)
    
    # Threshold distance transform to get foreground markers
    _, fg_markers_22bai1118 = cv2.threshold(dist_transform_22bai1118, 0.7 * dist_transform_22bai1118.max(), 255, 0)
    fg_markers_22bai1118 = np.uint8(fg_markers_22bai1118)
    
    # Background markers
    _, bg_markers_22bai1118 = cv2.threshold(dist_transform_22bai1118, 0.1 * dist_transform_22bai1118.max(), 255, 0)
    bg_markers_22bai1118 = np.uint8(bg_markers_22bai1118)
    
    # Unknown region
    unknown_22bai1118 = cv2.subtract(bg_markers_22bai1118, fg_markers_22bai1118)
    
    # Marker labelling
    _, markers_22bai1118 = cv2.connectedComponents(fg_markers_22bai1118)
    markers_22bai1118 = markers_22bai1118 + 1
    markers_22bai1118[unknown_22bai1118 == 255] = 0
    
    return markers_22bai1118

def watershed_segmentation_22bai1118(image_22bai1118, markers_22bai1118):
    # Compute gradient
    gradient_22bai1118 = filters.sobel(image_22bai1118)
    
    # Watershed transform
    labels_22bai1118 = watershed(gradient_22bai1118, markers_22bai1118, watershed_line=True)
    
    return labels_22bai1118


# Create tumor-specific markers
markers_22bai1118 = create_tumor_markers_22bai1118(preprocessed_22bai1118)

# Apply watershed segmentation
segmented_22bai1118 = watershed_segmentation_22bai1118(preprocessed_22bai1118, markers_22bai1118)

# Post-processing to refine tumor region
tumor_mask_22bai1118 = np.zeros_like(segmented_22bai1118)
tumor_mask_22bai1118[segmented_22bai1118 > 1] = 255
tumor_mask_22bai1118 = morph.remove_small_objects(tumor_mask_22bai1118.astype(bool), min_size=100)
tumor_mask_22bai1118 = morph.binary_closing(tumor_mask_22bai1118, morph.disk(3))

# Overlay segmentation result on original image
overlay_22bai1118 = cv2.cvtColor(image_22bai1118, cv2.COLOR_GRAY2RGB)
overlay_22bai1118[tumor_mask_22bai1118] = [255, 0, 0]  # Red color for tumor region

# Display results
plt.figure(figsize=(15, 5))
plt.subplot(131), plt.imshow(image_22bai1118, cmap='gray'), plt.title('Original Image')
plt.subplot(132), plt.imshow(segmented_22bai1118, cmap='nipy_spectral'), plt.title('Watershed Segmentation')
plt.subplot(133), plt.imshow(overlay_22bai1118), plt.title('Tumor Overlay')
plt.tight_layout()
plt.show()

# Save results
cv2.imwrite('segmented_tumor.png', (tumor_mask_22bai1118 * 255).astype(np.uint8))
cv2.imwrite('tumor_overlay.png', cv2.cvtColor(overlay_22bai1118, cv2.COLOR_RGB2BGR))
## Task 3: Graph Cut Algorithm Implementation

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters
from skimage.segmentation import watershed
from skimage import morphology as morph

preprocessed_22bai1118 = preprocess_image_22bai1118(image_22bai1118)

def create_tumor_markers_22bai1118(image_22bai1118):
    # Apply adaptive thresholding
    _, binary_22bai1118 = cv2.threshold(image_22bai1118, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphological operations to refine tumor region
    kernel_22bai1118 = np.ones((3,3), np.uint8)
    opening_22bai1118 = cv2.morphologyEx(binary_22bai1118, cv2.MORPH_OPEN, kernel_22bai1118, iterations=2)
    
    # Distance transform
    dist_transform_22bai1118 = cv2.distanceTransform(opening_22bai1118, cv2.DIST_L2, 5)
    
    # Threshold distance transform to get foreground markers
    _, fg_markers_22bai1118 = cv2.threshold(dist_transform_22bai1118, 0.7 * dist_transform_22bai1118.max(), 255, 0)
    fg_markers_22bai1118 = np.uint8(fg_markers_22bai1118)
    
    # Background markers
    _, bg_markers_22bai1118 = cv2.threshold(dist_transform_22bai1118, 0.1 * dist_transform_22bai1118.max(), 255, 0)
    bg_markers_22bai1118 = np.uint8(bg_markers_22bai1118)
    
    # Unknown region
    unknown_22bai1118 = cv2.subtract(bg_markers_22bai1118, fg_markers_22bai1118)
    
    # Marker labelling
    _, markers_22bai1118 = cv2.connectedComponents(fg_markers_22bai1118)
    markers_22bai1118 = markers_22bai1118 + 1
    markers_22bai1118[unknown_22bai1118 == 255] = 0
    
    return markers_22bai1118

def watershed_segmentation_22bai1118(image_22bai1118, markers_22bai1118):
    # Compute gradient
    gradient_22bai1118 = filters.sobel(image_22bai1118)
    
    # Watershed transform
    labels_22bai1118 = watershed(gradient_22bai1118, markers_22bai1118, watershed_line=True)
    
    return labels_22bai1118


# Create tumor-specific markers
markers_22bai1118 = create_tumor_markers_22bai1118(preprocessed_22bai1118)

# Apply watershed segmentation
segmented_22bai1118 = watershed_segmentation_22bai1118(preprocessed_22bai1118, markers_22bai1118)

# Post-processing to refine tumor region
tumor_mask_22bai1118 = np.zeros_like(segmented_22bai1118)
tumor_mask_22bai1118[segmented_22bai1118 > 1] = 255
tumor_mask_22bai1118 = morph.remove_small_objects(tumor_mask_22bai1118.astype(bool), min_size=100)
tumor_mask_22bai1118 = morph.binary_closing(tumor_mask_22bai1118, morph.disk(3))

# Overlay segmentation result on original image
overlay_22bai1118 = cv2.cvtColor(image_22bai1118, cv2.COLOR_GRAY2RGB)
overlay_22bai1118[tumor_mask_22bai1118] = [255, 0, 0]  # Red color for tumor region

# Display results
plt.figure(figsize=(15, 5))
plt.subplot(131), plt.imshow(image_22bai1118, cmap='gray'), plt.title('Original Image')
plt.subplot(132), plt.imshow(segmented_22bai1118, cmap='nipy_spectral'), plt.title('Watershed Segmentation')
plt.subplot(133), plt.imshow(overlay_22bai1118), plt.title('Tumor Overlay')
plt.tight_layout()
plt.show()

# Save results
cv2.imwrite('segmented_tumor.png', (tumor_mask_22bai1118 * 255).astype(np.uint8))
cv2.imwrite('tumor_overlay.png', cv2.cvtColor(overlay_22bai1118, cv2.COLOR_RGB2BGR))



